1. Estime os parâmetros $\beta_k$, $\beta_l$ por GMM utilizando o estimador apresentado acima. Escreva um programa em Matlab utilizando o solver fminsearch.

In [1]:
import pandas as pd

chilean = pd.read_csv('chilean.csv')


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

df = chilean.rename(columns={'Y': 'y', 'sX': 'k', 'pX': 'm', 'fX1': 'fx1', 'fX2': 'fx2'}).copy()


# l = ln(exp(fx1) + exp(fx2))
df['l'] = np.log(np.exp(df['fx1']) + np.exp(df['fx2']))

# Selecionar variáveis relevantes e criar uma cópia explícita
df_panel = df[['idvar', 'timevar', 'y', 'k', 'l', 'm']].copy()

# Ordenar dados para lagging
df_panel.sort_values(['idvar', 'timevar'], inplace=True)

# Primeiro Estágio: OLS

# Criar termos polinomiais de 4ª ordem para f^-1(k, l, m)
X_stage1 = pd.DataFrame(index=df_panel.index)
X_stage1['const'] = 1
X_stage1['k'] = df_panel['k']
X_stage1['l'] = df_panel['l']

# $\sum_{i=0}^4 \beta k^i * l^{4-i}$
X_stage1['l_4'] = df_panel['l']**4
X_stage1['k_l_3'] = df_panel['k'] * (df_panel['l']**3)
X_stage1['k2_l2'] = (df_panel['k']**2) * (df_panel['l']**2)
X_stage1['k3_l'] = (df_panel['k']**3) * df_panel['l']
X_stage1['k_4'] = df_panel['k']**4

# m^4 + k*m^3 + k^2*m^2 + k^3*m (k^4 já foi definido)
X_stage1['m_4'] = df_panel['m']**4
X_stage1['k_m_3'] = df_panel['k'] * (df_panel['m']**3)
X_stage1['k2_m2'] = (df_panel['k']**2) * (df_panel['m']**2)
X_stage1['k3_m'] = (df_panel['k']**3) * df_panel['m']

# m*l^3 + m^2*l^2 + m^3*l (l^4 e m^4 já foram definidos)
X_stage1['m_l_3'] = df_panel['m'] * (df_panel['l']**3)
X_stage1['m2_l2'] = (df_panel['m']**2) * (df_panel['l']**2)
X_stage1['m3_l'] = (df_panel['m']**3) * df_panel['l']

y_stage1 = df_panel['y']


data_stage1_full = pd.concat([y_stage1, X_stage1], axis=1)
data_stage1_clean = data_stage1_full.dropna()

y_stage1_clean = data_stage1_clean['y']
X_stage1_clean = data_stage1_clean.drop(columns=['y'])

model_stage1 = sm.OLS(y_stage1_clean, X_stage1_clean)
results_stage1 = model_stage1.fit()
print(results_stage1.summary())

beta_0_tilde = results_stage1.params['const']
print(f"\n Intercepto do primeiro estágio: {beta_0_tilde}")

# Calcular Phi_hat e adicionar de volta ao df_panel original usando o índice
df_panel['Phi_hat'] = np.nan
df_panel.loc[X_stage1_clean.index, 'Phi_hat'] = results_stage1.predict(X_stage1_clean)



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     843.9
Date:                Wed, 18 Jun 2025   Prob (F-statistic):               0.00
Time:                        13:38:09   Log-Likelihood:                -2369.4
No. Observations:                2544   AIC:                             4769.
Df Residuals:                    2529   BIC:                             4856.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.2094      0.140     65.592      0.0

In [ ]:
# Segundo Estágio: GMM

# Criar variáveis defasadas (k_{t-1}, l_{t-1}, Phi_hat_{t-1})
df_panel['k_lag1'] = df_panel.groupby('idvar')['k'].shift(1)
df_panel['l_lag1'] = df_panel.groupby('idvar')['l'].shift(1)
df_panel['Phi_hat_lag1'] = df_panel.groupby('idvar')['Phi_hat'].shift(1)

# Selecionar apenas as colunas necessárias para o GMM
df_stage2 = df_panel[['y', 'k', 'l', 'Phi_hat_lag1', 'k_lag1', 'l_lag1']].dropna()

y_jt = df_stage2['y'].values
k_jt = df_stage2['k'].values
l_jt = df_stage2['l'].values
Phi_hat_t_minus_1 = df_stage2['Phi_hat_lag1'].values
k_jt_minus_1 = df_stage2['k_lag1'].values
l_jt_minus_1 = df_stage2['l_lag1'].values

rho_hat = 0.9528 # Valor fixo conforme a questão

# Função de momentos para GMM
def gmm_moments(params_gmm):
    beta_k, beta_l = params_gmm
    
    # Resíduo u_jt da Equação (6)
    # u_jt = y_jt - beta_0_tilde - beta_k*k_jt - beta_l*l_jt - rho_hat * (Phi_hat_{t-1} - beta_0_tilde - beta_k*k_{t-1} - beta_l*l_{t-1})
    
    term_current_prod = y_jt - beta_0_tilde - beta_k * k_jt - beta_l * l_jt
    term_lagged_prod_expectation = Phi_hat_t_minus_1 - beta_0_tilde - beta_k * k_jt_minus_1 - beta_l * l_jt_minus_1
    
    u_jt = term_current_prod - rho_hat * term_lagged_prod_expectation
    
    # Condições de momento E[u_jt * Z_jt] = 0
    # Z_jt = [k_jt, l_jt_minus_1]
    moment1 = np.mean(u_jt * k_jt)           # Instrumento: k_jt
    moment2 = np.mean(u_jt * l_jt_minus_1)   # Instrumento: l_jt-1
    
    return np.array([moment1, moment2])

# Função objetivo para GMM (soma dos quadrados dos momentos)
# Para GMM exatamente identificado, a matriz de ponderação W pode ser a identidade.
def gmm_objective(params_gmm):
    moments = gmm_moments(params_gmm)
    return np.sum(moments**2) # Equivalente a m'I m

# Valores iniciais para beta_k, beta_l (podem vir de um OLS simples ou serem arbitrários razoáveis)
# Usando os coeficientes de k e l do primeiro estágio como ponto de partida pode ser uma boa ideia.
# No entanto, a questão implica que beta_k e beta_l do primeiro estágio são diferentes dos do segundo.
# Vamos usar valores iniciais comuns para elasticidades.
initial_params_gmm = [0.1, 0.6] 

print("\nSegundo Estágio - GMM:")
# Otimização usando Nelder-Mead (análogo ao fminsearch do Matlab)
result_gmm = minimize(gmm_objective, initial_params_gmm, method='Nelder-Mead')

print("Optimization successful.")
beta_k_gmm, beta_l_gmm = result_gmm.x
print(f"   Final GMM objective function value: {result_gmm.fun:.6e}")
print(f"   Estimated beta_k: {beta_k_gmm:.4f}")
print(f"   Estimated beta_l: {beta_l_gmm:.4f}")


2. Calcule a estatística de Wald (F)